In [16]:
import os
from symmer import PauliwordOp, QuantumState, QubitTapering
from symmer.operators import AntiCommutingOp, single_term_expval
from symmer.utils import exact_gs_energy
from symmer.evolution.exponentiation import *
from symmer.evolution.decomposition import *
import json
import numpy as np
from vqe.measurements import measureObservable
from tomography.cst import ClassicalShadow
from qiskit import QuantumCircuit
from qiskit.extensions import UnitaryGate

# Set up directories
cwd = os.getcwd()
file_dir =  os.path.dirname(cwd)
symmer_dir = os.path.join(cwd, 'symmer')
test_dir = os.path.join(symmer_dir, 'tests')
ham_data_dir = os.path.join(test_dir, 'hamiltonian_data')

# Get a Hamiltonian
ham_list = os.listdir(ham_data_dir)
num_qubits = []
for ham in ham_list:
    with open(os.path.join(ham_data_dir, ham), 'r') as infile:
        data_dict = json.load(infile)
    H = PauliwordOp.from_dictionary(data_dict['hamiltonian'])
    num_qubits.append(H.n_qubits)
sorted_list = sorted(zip(ham_list, num_qubits), key = lambda x: x[1])
ham_list = [ham for ham,_ in sorted_list]

In [17]:
filename = ham_list[0]
print(filename)
with open(os.path.join(ham_data_dir, filename), 'r') as infile:
    data_dict = json.load(infile)
H = PauliwordOp.from_dictionary(data_dict['hamiltonian'])

# Tapering
QT = QubitTapering(H)
hf_state   = QuantumState(np.asarray(data_dict['data']['hf_array'])) # Hartree-Fock state
hf_energy  = data_dict['data']['calculated_properties']['HF']['energy']
H_taper   = QT.taper_it(ref_state=hf_state) 

# Exact Energy
gs_nrg_tap, gs_psi_tap = exact_gs_energy(H_taper.to_sparse_matrix)
print("The exact gs energy is " + str(gs_nrg_tap))

# Check that gs_psi_tap is the true gs
nrg_calc = 0
for p in range(H_taper.n_terms):
    nrg_calc += H_taper.coeff_vec[p] * single_term_expval(H_taper[p], gs_psi_tap)
print(nrg_calc)

# Get qiskit circuit preparing gs_psi_tap
sp_circ = QuantumCircuit(gs_psi_tap.n_qubits)
gs_psi_tap_dict = gs_psi_tap.to_dictionary
gs_psi_tap_array = np.array([0]*(2**gs_psi_tap.n_qubits), dtype=complex)
def bin_to_int(b):
    b = b[::-1]
    l = len(b)
    val = 0
    for a in range(l):
        val += int(b[a]) * (2**(l-1-a))
    return val
for p, w in zip(list(gs_psi_tap_dict.keys()), list(gs_psi_tap_dict.values())):
    val = bin_to_int(p)
    gs_psi_tap_array[val] = w 
sp_circ.prepare_state(gs_psi_tap_array)
print(gs_psi_tap)

H3+_STO-3G_SINGLET_JW.json
The exact gs energy is -1.274412699726855
(-1.274412699726855+0j)
-0.991+0.000j |000> +
-0.097+0.000j |001> +
 0.000+0.000j |010> +
 0.000+0.000j |100> +
 0.097+0.000j |110>


In [18]:
# Run a quantum experiment to estimate gs energy given gs_psi_tap and H_taper
sp_circ_copy = sp_circ.copy()
gs_nrg_basic_vqe = measureObservable(sp_circ_copy, range(sp_circ_copy.num_qubits), H_taper, shots=1000, total_shots_provided=True)
print("basic VQE: " + str(gs_nrg_basic_vqe))

basic VQE: (-1.2501235902299617+0j)


In [19]:
# Run a quantum experiment to estimate gs energy given gs_psi_tap and H_taper using unitary partitioning
clique_cover = H_taper.clique_cover(edge_relation='AC')
gs_nrg_up_vqe = 0
for clique in clique_cover.values():
    ac_op = AntiCommutingOp.from_PauliwordOp(clique)
    pop,rots,w,op = ac_op.unitary_partitioning()
    sp_circ_copy = sp_circ.copy()
    if rots != None:
          rots = rots[::-1]
          for rot, coeff in rots:
            rot = rot * -coeff
            rot = PauliwordOp_to_QuantumCircuit(rot)           
            sp_circ_copy = sp_circ_copy.compose(rot, list(range(sp_circ_copy.num_qubits)))
    gs_nrg_up_vqe += w * measureObservable(sp_circ_copy, range(sp_circ_copy.num_qubits), pop, shots=1000, total_shots_provided=True)
print("UP VQE: " + str(gs_nrg_up_vqe))

UP VQE: (1.2994538926727897+0j)


In [20]:
# Estimate gs energy given gs_psi_tap and H_taper using classical shadows
sp_circ_copy = sp_circ.copy()
H_taper_dict = H_taper.to_dictionary
obs = list(H_taper_dict.keys())
classical_shadow = ClassicalShadow(sp_circ_copy, obs)
classical_shadow.createClassicalShadows(unitary_ensemble="random clifford", num_shadows=1000)

snapshots collected
shadows obtained


In [21]:
classical_shadow.observables = [PauliwordOp.from_dictionary({o : w}).to_sparse_matrix for o,w in zip(list(H_taper_dict.keys()), list(H_taper_dict.values()))]
obs, results = classical_shadow.linearPredictions(10)
gs_nrg_tap_cst = 0
for w, exp in zip(list(H_taper_dict.values()), results):
    gs_nrg_tap_cst += exp 
print("Classical shadows: " + str(gs_nrg_tap_cst))

Classical shadows: (-1.1458542834262224+0j)


In [22]:
# Estimate gs energy given gs_psi_tap and H_taper using unitary partitioning and classical shadows
classical_shadow.observables = []
weights = []
for clique in clique_cover.values():
    ac_op = AntiCommutingOp.from_PauliwordOp(clique)
    pop,rots,w,op = ac_op.unitary_partitioning()
    classical_shadow.observables.append(op.to_sparse_matrix)
    weights.append(w)
_, results = classical_shadow.linearPredictions(10)
gs_nrg_tap_cst_up = 0
for w, exp in zip(weights, results):
    gs_nrg_tap_cst_up += w * exp 
print("Classical shadows and UP: " + str(gs_nrg_tap_cst_up))

Classical shadows and UP: (-1.1334160766999342-1.0476469810368112e-18j)


In [23]:
# Estimate gs energy given gs_nrg_tap and H_taper using classical shadows in the Pauli basis
sp_circ_copy = sp_circ.copy()
H_taper_dict = H_taper.to_dictionary
obs = list(H_taper_dict.keys())
classical_shadow = ClassicalShadow(sp_circ_copy, obs)
classical_shadow.createClassicalShadows(unitary_ensemble="pauli", num_shadows=1000)

snapshots collected
shadows obtained


In [24]:
classical_shadow.observables = [PauliwordOp.from_dictionary({o : w}).to_sparse_matrix for o,w in zip(list(H_taper_dict.keys()), list(H_taper_dict.values()))]
obs, results = classical_shadow.linearPredictions(10)
gs_nrg_tap_cst = 0
for w, exp in zip(list(H_taper_dict.values()), results):
    gs_nrg_tap_cst += exp 
print("Classical shadows Pauli: " + str(gs_nrg_tap_cst))

Classical shadows Pauli: (-3.220654977213096+0j)


In [25]:
# Estimate gs energy given gs_psi_tap and H_taper using unitary partitioning and classical shadows
classical_shadow.observables = []
weights = []
for clique in clique_cover.values():
    ac_op = AntiCommutingOp.from_PauliwordOp(clique)
    pop,rots,w,op = ac_op.unitary_partitioning()
    classical_shadow.observables.append(op.to_sparse_matrix)
    weights.append(w)
_, results = classical_shadow.linearPredictions(10)
gs_nrg_tap_cst_up = 0
for w, exp in zip(weights, results):
    gs_nrg_tap_cst_up += w * exp 
print("Classical shadows Pauli and UP: " + str(gs_nrg_tap_cst_up))

Classical shadows Pauli and UP: (-3.2200886468021697-1.4706965387688823e-18j)
